In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd
import pickle

In [2]:
url = 'https://www.beeradvocate.com/beer/profile/1170/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [100]:
def get_brewery_info(brew_nums):
    
    dic = {}
    
    for brew_num in brew_nums:
        url = 'https://www.beeradvocate.com/beer/profile/' + str(brew_num) + '/'
    
        # Getting soup

        key = []
        values = []
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")

        # Finding feature values

        try:
            title_div = soup.find(class_='titleBar')
            brew_name = str(title_div.text).replace('\n','')
        except:
            brew_name = 'NA'

        try:
            score_obj = soup.find('span', {'class': 'BAscore_big'})
            score = re.split(r'>|<', str(score_obj))[4]
        except:
            score = 'NA'

        try:   
            beer_class_obj = soup.find_all('b')[4]
            beer_class = re.split(r'>|<', str(beer_class_obj))[2]
        except:
            beer_class = 'NA'

        try:
            stats_objs = soup.find_all('dd')[5:13]
            stats_list = []
            for item in stats_objs:
                item = str(item.text).strip()
                if len(item) > 0:
                    clean_item = re.sub(r'#|%|,|/5','',item)
                    if '.' in clean_item:
                        stats_list.append(clean_item)
                    else:
                        stats_list.append(clean_item)
                else:
                    pass
            num_beers, num_beer_reviews, num_beer_ratings, brew_score, reviews, ratings, pDev = stats_list
        except:
            num_beers = soup.find_all('dd')[5].text
            num_beer_reviews = soup.find_all('dd')[6].text
            num_beer_ratings = soup.find_all('dd')[7].text
            
            stats_list = ('NA '*4).split()

            brew_score, reviews, ratings, pDev = stats_list

        # Lots of data floats freely within info_box

        info_box = soup.find('div',attrs={'id':'info_box'})

        try:
            b_type = re.split(r'INFO</b>\n<br/><br/>\n<b>Type:</b> |\n<br/><br/>\n',
                     str(info_box),maxsplit=4)[1]
        except:
            b_type = 'NA'

        try:
            phone_string = re.split(r'</a><br/><br/>',
                                    str(info_box),maxsplit=4)
            phone_num_string = phone_string[1].split(' |')[0]
            phone_num = re.sub(r'\(|\)| |-','',phone_num_string)
            int(phone_num)
            has_phone = True
        except:
            has_phone = False

        try:   
            comm_desc_str = str(info_box).split('Notes:</b>\n<br/>\n')[1]
            comm_desc = comm_desc_str.split('<br')[0]
        except:
            comm_desc = 'NA'        

        key = brew_name

        values = [score, beer_class, num_beers, 
                  num_beer_reviews, num_beer_ratings, brew_score, 
                  reviews, ratings, pDev, b_type, has_phone, comm_desc]
        
        dic[key] = values
    
        time.sleep(np.random.poisson(10)/2)

    
    return dic

In [101]:
get_brewery_info([45825])
# 35989

{'Perle - Bières Artisanales Artzner': ['3.67',
  'Good',
  '11',
  '0',
  '5',
  'NA',
  'NA',
  'NA',
  'NA',
  'Brewery',
  True,
  'No notes at this time.']}

In [20]:
info_box = soup.find('div',attrs={'id':'info_box'})

In [ ]:
    stats_objs = soup.find_all('dd')[5:13]
    stats_list = []
    for item in stats_objs:
        item = str(item.text).strip()
        if len(item) > 0:
            clean_item = re.sub(r'#|%|,|/5','',item)
            if '.' in clean_item:
                stats_list.append(clean_item)
            else:
                stats_list.append(clean_item)
        else:
            pass

    if len(stats_list) < 7:
        stats_list = ('NA '*7).split()

    ranking, reviews, ratings, pDev, wants, gots, trade = stats_list

In [81]:
stats_string = soup.find_all('dd')[5:13]
stats_string

#phone_string = re.split(r'</a><br/><br/>',
#         str(info_box),maxsplit=4)
#phone_num_string = phone_string[1].split(' |')[0]
#phone_num = re.sub(r'\(|\)| |-','',phone_num_string)
#int(phone_num)

[<dd>116</dd>,
 <dd>2,581</dd>,
 <dd>9,967</dd>,
 <dd><a class="ba-score" href="/beer/profile/1170/?view=ratings">4.03/5</a></dd>,
 <dd><span class="ba-reviews">58</span></dd>,
 <dd><span class="ba-ratings">138</span></dd>,
 <dd><span class="ba-pdev">
 10.17% </span></dd>,
 <dd><input class="textCtrl" id="LoginControl" name="login" tabindex="101" type="text"/></dd>]

In [82]:
url = 'https://www.beeradvocate.com/beer/profile/45825/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [83]:
stats_string = soup.find_all('dd')[5:13]
stats_string

[<dd>11</dd>,
 <dd>0</dd>,
 <dd>5</dd>,
 <dd><input class="textCtrl" id="LoginControl" name="login" tabindex="101" type="text"/></dd>,
 <dd>
 <ul>
 <li><label for="ctrl_not_registered"><input id="ctrl_not_registered" name="register" tabindex="105" type="radio" value="1"/>
 No, create an account now.</label></li>
 <li><label for="ctrl_registered"><input checked="checked" class="Disabler" id="ctrl_registered" name="register" tabindex="105" type="radio" value="0"/>
 Yes, my password is:</label></li>
 <li id="ctrl_registered_Disabler">
 <input class="textCtrl" id="ctrl_password" name="password" tabindex="102" type="password"/>
 <div class="lostPassword"><a class="OverlayTrigger OverlayCloser" href="lost-password/" tabindex="106">Forgot your password?</a></div>
 </li>
 </ul>
 </dd>,
 <dd>
 <input class="button primary" data-loginphrase="Log in" data-signupphrase="Sign up" tabindex="104" type="submit" value="Log in"/>
 <label class="rememberPassword" for="ctrl_remember"><input id="ctrl_remem

In [93]:
beer_class_obj = soup.find_all('b')[4]
beer_class = re.split(r'>|<', str(beer_class_obj))[2]
beer_class

'Good'

In [86]:
ranking = soup.find_all('dd')[5].text
ranking

'11'

In [87]:
reviews = soup.find_all('dd')[6].text
reviews

'0'

In [89]:
ratings = soup.find_all('dd')[7].text
ratings

'5'

In [ ]:
ranking = soup.find_all('dd')[5].text
reviews = soup.find_all('dd')[6].text
ratings = soup.find_all('dd')[7].text

In [46]:
# str(abv_div).split('Availability:</b> ')[1]
phone_string = re.split(r'</a><br/><br/>',
         str(info_box),maxsplit=4)
phone_num_string = phone_string[1].split(' |')[0]
phone_num = re.sub(r'\(|\)| |-','',phone_num_string)
int(phone_num)

8123362337

In [76]:
df = pd.read_pickle('140_13550')

In [77]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Research Series: Ea-kwa-Yes (8/31/18) | Frost Beer Works,0,,-,0,0,0,0,0,0,Frost Beer Works,Vermont,United States,American Imperial IPA,8.00,Limited (brewed once),No notes at this time.,10-25-2018
Hammer And Ale | Two Brothers Brewing Company,0,,-,0,0,0,0,0,0,Two Brothers Brewing Company,Illinois,United States,American Imperial IPA,not,Rotating,No notes at this time.,08-11-2015
Flower Fury | North Slope Brewing Company,0,,-,0,0,0,0,0,0,North Slope Brewing Company,Pennsylvania,United States,American Imperial IPA,10.0,Rotating,No notes at this time.,08-22-2015
Tequila Barrel Aged Agave IPA | Local Relic,0,,-,0,0,0,0,0,0,Local Relic,Colorado,United States,American Imperial IPA,9.10,Rotating,No notes at this time.,12-10-2015
Crimethink | Zagovor Brewery,0,,-,0,0,0,0,0,0,Zagovor Brewery,Russian Federation,zagovorbrewery.com,American Imperial IPA,9.70,Year-round,No notes at this time.,12-16-2015
Orbz | Crooked Run Central,0,,-,0,0,0,0,0,0,Crooked Run Central,Virginia,United States,American Imperial IPA,8.50,Rotating,No notes at this time.,07-26-2018
Imperial IPA | Cervejas Krios,0,,-,0,0,0,0,0,0,Cervejas Krios,Brazil,American Imperial IPA,American Imperial IPA,7.50,Year-round,No notes at this time.,12-07-2017
Juwanna IPA? | City Steam Brewery Café,0,,-,0,0,0,0,0,0,City Steam Brewery Café,Connecticut,United States,American Imperial IPA,10.0,Rotating,No notes at this time.,12-13-2017
Golden Sword | Prairie Krafts Brewing Company,0,,-,0,0,0,0,0,0,Prairie Krafts Brewing Company,Illinois,United States,American Imperial IPA,8.00,Rotating,No notes at this time.,06-04-2017
Hopstrong | Amundsen Bryggeri & Spiseri,0,,-,0,0,0,0,0,0,Amundsen Bryggeri & Spiseri,Norway,amundsenbryggeri.no,American Imperial IPA,8.50,Rotating,No notes at this time.,06-03-2015
